## desca los audio s


In [ ]:
import os
import requests
import pandas as pd

# ---- 1) Leer el CSV correctamente (separador ;) ----
csv_path = df
#df = pd.read_csv(csv_path, sep=';', dtype=str)   # <<< clave
df.columns = df.columns.str.strip().str.lower()

print(df.head())
print(df.columns.tolist())  # debería mostrar ['call_id', 'recording_url']

# ---- 2) Carpeta de salida ----
out_dir = r"C:\Users\HAIDER_BELLO\Downloads\audios_2" # ruta donde se guardarán los audios
os.makedirs(out_dir, exist_ok=True)

# ---- 3) Descargar usando call_id como nombre ----
for _, row in df.iterrows():
    call_id = row['call_id'].strip()
    url = row['recording_url'].strip()

    # intenta conservar la extensión del link (wav)
    ext = '.' + url.split('?')[0].split('.')[-1] if '.' in url.split('?')[0] else '.wav'
    filepath = os.path.join(out_dir, f"{call_id}{ext}")

    try:
        print(f"Descargando {call_id} ...")
        r = requests.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(filepath, "wb") as f:
            for chunk in r.iter_content(8192):
                if chunk:
                    f.write(chunk)
        print(f"✅ Guardado: {filepath}")
    except Exception as e:
        print(f"❌ Error con {call_id}: {e}")


                                call_id  \
0  b0e8eb74-7b29-4659-b1b2-99d3d5de475f   
1  ff06c0ab-25b3-4805-922e-3148a64bf936   
2  da0e0d7d-348b-4e71-a539-c4c9243f2e4e   
3  ada6c1a3-6e71-4047-bd96-8a1e0e30e14a   
4  3b76ff72-d713-4407-94c1-7e55807b05ec   

                                       recording_url  
0  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
1  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
2  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
3  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
4  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
['call_id', 'recording_url']
Descargando b0e8eb74-7b29-4659-b1b2-99d3d5de475f ...
✅ Guardado: C:\Users\HAIDER_BELLO\Downloads\audios_2\b0e8eb74-7b29-4659-b1b2-99d3d5de475f.wav
Descargando ff06c0ab-25b3-4805-922e-3148a64bf936 ...
✅ Guardado: C:\Users\HAIDER_BELLO\Downloads\audios_2\ff06c0ab-25b3-4805-922e-3148a64bf936.wav
Descargando da0e0d7d-348b-4e71-a539-c4c9243f2e4e ...
✅ Guardado: C:\Users\HAIDER_BELLO\D

## verificar si ya estan descargados  

In [ ]:
import os, time, requests, pandas as pd
from urllib.parse import urlparse

# === Configura rutas ===
csv_path = df
out_dir  = r"C:\Users\haider_bello\Downloads\audios_2"
os.makedirs(out_dir, exist_ok=True)

# === Leer CSV correctamente (separador ;) y como texto) ===
df = pd.read_csv(
    csv_path,
    sep=';',
    dtype={'call_id': str, 'recording_url': str},
    keep_default_na=False,   # no conviertas "NA", "NaN" a NaN
    na_filter=False          # desactiva detección automática de NaN
)
df.columns = df.columns.str.strip().str.lower()

# === Limpiar datos ===
df['call_id'] = df['call_id'].astype(str).str.strip()
df['recording_url'] = df['recording_url'].astype(str).str.strip()

# Filtrar filas válidas
valid = df['call_id'].ne('') & df['recording_url'].str.startswith(('http://','https://'))
bad = df[~valid]
df = df[valid].copy()

# Quitar duplicados por call_id; si hay, nos quedamos con el primero
dups = df[df.duplicated('call_id', keep='first')]
df = df.drop_duplicates('call_id', keep='first')

print(f"Total filas en CSV: {len(valid) + len(bad)}")
print(f"Válidas: {len(df)}  ·  Inválidas/omitidas: {len(bad)}  ·  Duplicadas call_id: {len(dups)}")

# === Descarga con reintentos simples ===
session = requests.Session()
timeout = 60
retries = 3

ok, fail = 0, 0
skipped_exists = 0

for _, row in df.iterrows():
    call_id = row['call_id']
    url = row['recording_url']

    # Detectar extensión del archivo
    path_part = urlparse(url).path
    ext = os.path.splitext(path_part)[1] or '.wav'
    fname = f"{call_id}{ext}"
    fpath = os.path.join(out_dir, fname)

    if os.path.exists(fpath):
        skipped_exists += 1
        continue

    success = False
    for attempt in range(1, retries+1):
        try:
            r = session.get(url, stream=True, timeout=timeout)
            r.raise_for_status()
            with open(fpath, 'wb') as f:
                for chunk in r.iter_content(1024*1024):
                    if chunk:
                        f.write(chunk)
            success = True
            ok += 1
            break
        except Exception as e:
            if attempt < retries:
                time.sleep(2 * attempt)  # backoff
            else:
                print(f"❌ {call_id} ({url[:80]}...): {e}")
                fail += 1

print("\nResumen:")
print(f"  ✅ Descargados nuevos: {ok}")
print(f"  ⏭️  Ya existían (omitidos): {skipped_exists}")
print(f"  ❌ Fallidos: {fail}")
print(f"  🧹 Filas inválidas (vacías/sin http): {len(bad)}")
print(f"  ♻️  Duplicados de call_id omitidos: {len(dups)}")


Total filas en CSV: 565
Válidas: 563  ·  Inválidas/omitidas: 1  ·  Duplicadas call_id: 0

Resumen:
  ✅ Descargados nuevos: 207
  ⏭️  Ya existían (omitidos): 356
  ❌ Fallidos: 0
  🧹 Filas inválidas (vacías/sin http): 1
  ♻️  Duplicados de call_id omitidos: 0


In [11]:
!pip install faster-whisper


  Using cached ctranslate2-4.6.0-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.0/1.1 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------- 1


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121



Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 3.4 MB/s eta 0:11:58
     ---------------------------------------- 0.0/2.4 GB 2.3 MB/s eta 0:17:40
     ---------------------------------------- 0.0/2.4 GB 3.3 MB/s eta 0:12:25
     ---------------------------------------- 0.0/2.4 GB 4.1 MB/s eta 0:10:02
     ---------------------------------------- 0.0/2.4 GB 5.3 MB/s eta 0:07:38
     ---------------------------------------- 0.0/2.4 GB 7.2 MB/s eta 0:05:38
     ---------------------------------------- 0.0/2.4 GB 9.9 MB/s eta 0:04:08
     ---------------------------------------- 0.0/2.4 GB 12.9 MB/s eta 0:03:10
     ---------------------------------------- 0.0/2.4 GB 15.3 MB/s eta 0:02:40
     ---------------------------------------- 0.0/2.4 GB 18.5 MB/s eta 0:02:13
     ---------------------------------------- 0.0/2.4 GB 32.7 MB/s eta 0:01:15
     ---


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## consulta de sql para fechas y link de los audios entrenador

In [ ]:
import pyodbc
import pandas as pd

# Parámetros de conexión
server = '172.16.1.33'
database = 'CUN_REPOSITORIO'

try:
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )

    # Consulta con filtro por fecha
    query = """
    SELECT 
call_id 
,recording_url
FROM CUN_REPOSITORIO.dbo.vapi_entrenador
WHERE started_at >= CONVERT(datetime2(7), '2025-09-14T00:00:00', 126) --fecha de inicio
  AND started_at <  CONVERT(datetime2(7), '2025-10-01T00:00:00', 126) -- fecha final
ORDER BY started_at;
    
    
    """

    # Ejecutar y guardar en DataFrame
    df = pd.read_sql(query, conn)
    print(df)

    conn.close()

except Exception as e:
    print("❌ Error en la conexión o consulta:", e)

                                  call_id  \
0    b0e8eb74-7b29-4659-b1b2-99d3d5de475f   
1    ff06c0ab-25b3-4805-922e-3148a64bf936   
2    da0e0d7d-348b-4e71-a539-c4c9243f2e4e   
3    ada6c1a3-6e71-4047-bd96-8a1e0e30e14a   
4    3b76ff72-d713-4407-94c1-7e55807b05ec   
..                                    ...   
153  0199977e-b910-711e-a9b2-46ec1c2caf00   
154  01999780-3d77-7881-bc19-caa50e4df6a3   
155  01999782-38ea-7bb3-8cab-9b7a0eee0f05   
156  01999783-e52b-7bbb-bb9f-2ff1ca285968   
157  01999cae-a971-7998-9459-476aef1c6e85   

                                         recording_url  
0    https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
1    https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
2    https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
3    https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
4    https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
..                                                 ...  
153  https://guard-voz-ia.s3.us-east-1.amazonaws.co...  
154

C:\Users\haider_bello\AppData\Local\Temp\ipykernel_3796\305857972.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## los convirete en txt 

In [1]:
# !pip install faster-whisper torch --upgrade
import os
import time
import math
import threading
from queue import Queue
from pathlib import Path
from faster_whisper import WhisperModel
import torch

# ───── RUTAS (AJUSTA) ───────────────────────────────────────────────────────────
INPUT_ROOT = Path(r"C:\Users\haider_bello\Downloads\renombrado_entrante-20251002T210159Z-1-001\renombrado_entrante")
OUTPUT_DIR = Path(r"C:\Users\haider_bello\Downloads\renombrado_entrante-20251002T210159Z-1-001\renombrado_entrante\transcripciones")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

AUDIO_EXTS = {".wav", ".mp3", ".m4a", ".flac", ".ogg", ".aac", ".wma"}

# ───── DETECCIÓN GPU / CONFIG ──────────────────────────────────────────────────
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Para 90% de uso necesitas GPU. No se detectó CUDA."
torch.set_float32_matmul_precision("medium")

MODEL_NAME   = "medium"
COMPUTE_TYPE = "float16"           # FP16 en GPU = rápido
BEAM_SIZE    = 1                   # 1 = más veloz
LANG         = "es"

# Estimar cuántas instancias caben en 12 GB (aprox medium FP16 ~4-5 GB)
free, total = torch.cuda.mem_get_info()
free_gb  = free / (1024**3)
total_gb = total / (1024**3)

# Máximo 3 modelos; si VRAM libre <7.5 GB, usa 2; si >=11 GB, prueba 3
MAX_MODELS = 3 if free_gb >= 11 else (2 if free_gb >= 7.5 else 1)
print(f"GPU VRAM libre: {free_gb:.1f} GB / {total_gb:.1f} GB → instancias: {MAX_MODELS}")

# ───── UTILIDADES ───────────────────────────────────────────────────────────────
def gather_audio_files(root: Path):
    for dp, _, files in os.walk(root):
        for f in files:
            if Path(f).suffix.lower() in AUDIO_EXTS:
                yield Path(dp) / f

def transcribe_with_model(model: WhisperModel, audio_path: Path, out_dir: Path) -> str:
    out_path = out_dir / (audio_path.stem + ".txt")
    if out_path.exists():
        return f"➡️ {audio_path.name} (ya existe)"
    segments, _ = model.transcribe(
        str(audio_path),
        language=LANG,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=400),
        beam_size=BEAM_SIZE,
        best_of=1,
        temperature=0.0,
        without_timestamps=True,
        word_timestamps=False,
        condition_on_previous_text=False,
    )
    text = "".join(s.text for s in segments).strip()
    out_path.write_text(text, encoding="utf-8")
    return f"✔️ {audio_path.name} ({len(text)} chars)"

# ───── POOL DE WORKERS (CADA UNO CON SU MODELO) ────────────────────────────────
def worker_loop(worker_id: int, q: Queue, results: list, lock: threading.Lock):
    # Cada worker carga SU propia instancia del modelo (mantiene GPU ocupada)
    model = WhisperModel(
        MODEL_NAME,
        device="cuda",
        compute_type=COMPUTE_TYPE,
        device_index=0,      # cambia si usas otra GPU
        num_workers=1        # hilos internos del modelo (no subir en GPU)
    )
    while True:
        audio_path = q.get()
        if audio_path is None:
            q.task_done()
            break
        try:
            msg = transcribe_with_model(model, audio_path, OUTPUT_DIR)
        except Exception as e:
            msg = f"❌ {audio_path.name} → {e}"
        with lock:
            results.append(msg)
            print(msg)
        q.task_done()

def main():
    files = list(gather_audio_files(INPUT_ROOT))
    total = len(files)
    if total == 0:
        print("⚠️ No se encontraron audios.")
        return

    # Ordenar por tamaño descendente para balancear carga (los largos primero)
    files.sort(key=lambda p: p.stat().st_size if p.exists() else 0, reverse=True)

    print(f"🗂️  Audios: {total}  ·  Workers(modelos): {MAX_MODELS}\n")

    q = Queue(maxsize=MAX_MODELS * 4)  # prefetched items
    results, lock = [], threading.Lock()
    threads = []

    # Lanzar N workers (cada uno con su modelo)
    for i in range(MAX_MODELS):
        t = threading.Thread(target=worker_loop, args=(i, q, results, lock), daemon=True)
        t.start()
        threads.append(t)

    # Ingesta en la cola con barra de progreso simple
    start = time.time()
    for idx, f in enumerate(files, 1):
        q.put(f)
        if idx % 50 == 0:
            elapsed = time.time() - start
            done = len(results)
            if done:
                rate = done / elapsed               # archivos/seg
                eta  = (total - done) / rate
                print(f"⏳ {done}/{total} · {rate*60:.1f} a/min · ETA {eta/60:.1f} min")

    # Señales de fin
    for _ in range(MAX_MODELS):
        q.put(None)

    q.join()  # esperar a que terminen
    dur = time.time() - start
    print(f"\n🏁 Listo: {len(results)}/{total} en {dur/60:.1f} min "
          f"({(total/(dur/60)):0.1f} audios/min aprox)")

if __name__ == "__main__":
    main()


GPU VRAM libre: 10.9 GB / 12.0 GB → instancias: 2
🗂️  Audios: 1084  ·  Workers(modelos): 2

➡️ 3202416736_2025_09_03_11_30_15_Dayana_Alexandra_Marulanda_Feo.WAV (ya existe)
➡️ 3002494888_2025_09_11_07_42_59_Edwin_Cendales.WAV (ya existe)
✔️ 3197236622_2025_09_11_22_38_56_Angie_Daniela_Lancheros.WAV (10948 chars)
✔️ 6013019106_2025_09_11_18_00_01_Johan_Casallas.WAV (3415 chars)
✔️ 3156808774_2025_09_23_08_46_52_Angie_Daniela_Lancheros.WAV (16158 chars)
✔️ 3162458396_2025_09_01_10_21_53_Luisa_Reyes.WAV (4329 chars)
✔️ 3161979668_2025_09_10_09_59_54_Nicolas_Steven_Tovar_Zamudio.WAV (9542 chars)
✔️ 3209804625_2025_09_12_07_44_05_Edwin_Cendales.WAV (5717 chars)
✔️ 3106320421_2025_09_23_15_45_57_Edwin_Cendales.WAV (6016 chars)
✔️ 3185212004_2025_09_11_09_07_18_Edwin_Cendales.WAV (2796 chars)
✔️ 3144541264_2025_09_12_18_37_05_Luisa_Reyes.WAV (1679 chars)
✔️ 3112466380_2025_09_22_15_33_29_Edwin_Cendales.WAV (4213 chars)
✔️ 3046757735_2025_09_08_18_42_15_Luisa_Reyes.WAV (6386 chars)
✔️ 31748295

In [1]:
!pip uninstall -y faster-whisper ctranslate2
!pip install --upgrade --index-url https://download.pytorch.org/whl/cu121 torch
!pip install --upgrade ctranslate2-cu121
!pip install --upgrade ctranslate2-cuda12x




Found existing installation: faster-whisper 1.2.0
Uninstalling faster-whisper-1.2.0:
  Successfully uninstalled faster-whisper-1.2.0
Found existing installation: ctranslate2 4.6.0
Uninstalling ctranslate2-4.6.0:
  Successfully uninstalled ctranslate2-4.6.0
Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement ctranslate2-cu121 (from versions: none)
ERROR: No matching distribution found for ctranslate2-cu121

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement ctranslate2-cuda12x (from versions: none)
ERROR: No matching distribution found for ctranslate2-cuda12x

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m 

In [2]:
# Torch con CUDA 12.x (Windows)
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cu121
!pip install --upgrade faster-whisper


Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\haider_bello\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# -*- coding: utf-8 -*-
"""
Pipeline de evaluación de .txt + merge con metadatos (call_id, fecha, numero_vapi, numero10_vapi)
Requisitos:
  pip install pandas textblob pyodbc openpyxl
"""

from __future__ import annotations
import os, re, unicodedata
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd

# ===================== CONFIG =====================
# Carpeta de entrada con .txt
INPUT_TXT_DIR = r"C:\Users\haider_bello\Downloads\audios_2"
# Carpeta de salida
OUTPUT_DIR    = r"C:\Users\haider_bello\Downloads\audios_2\salidas"

# --- META DESDE SQL SERVER ---
SQL_SERVER = "172.16.1.33"
SQL_DB     = "CUN_REPOSITORIO"
# Autenticación integrada:
SQL_CONNSTR = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={SQL_SERVER};DATABASE={SQL_DB};Trusted_Connection=yes;"
)

# Filtro opcional de fechas del meta (semi-abierto). Deja None para no filtrar.
META_DESDE     = None            # ej: "2025-09-14"
META_HASTA_EXC = None            # ej: "2025-10-01"

# Tipo marcador de salida
TIPO = "entrenador"

# Si no hay fecha en meta para algún call_id, se usa esta por defecto:
FECHA_ANALISIS_DEF = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# ==================================================


# ====== DICCIONARIOS DE CATEGORÍAS  ====
DICCIONARIOS_KEYWORDS: Dict[str, List[str]] = {
    "saludo": [
        "alo","hola","buenos dias","buenas tardes","buenas noches","digame","si","quien habla","con quien hablo",
        "quien llama","de parte de quien","le puedo ayudar en algo","en que le puedo servir","que desea","que necesita",
        "habla con","soy","aqui","departamento de","le atiende","si le escucho","le oigo","adelante","te escucho","todo bien",
        "que tal","como le va","como te va","que cuentas","que hay de nuevo","que pasa","que me cuentas","a quien busca",
        "a quien desea hablar","con quien gustaria hablar","con quien le gustaria hablar","con quien te gustaria hablar","a quien busca por favor",
        "podria decirme con quien desea hablar","hablo con","si soy yo","en que puedo ayudarle","si habla con el","si habla con ella","si que necesita",
        "un segundo por favor","espere un momentito","ya le pongo","me puede repetir su nombre por favor","podria deletrear su nombre","cual es su apellido",
        "de que empresa llama","cual es su extension","no le escucho bien podria hablar mas alto","se oye entrecortado","puede repetir",
        "entendio","esta claro","perfecto","muy bien","de acuerdo","entendido","gracias","muchas gracias","a usted","con gusto","que tenga un buen dia",
        "que tenga buena tarde","que tenga buena noche","igualmente","hasta luego","adios","nos vemos","que le vaya bien","saludos","un placer atenderle",
        "gracias por su tiempo","nombre","area","admisiones","interes","acompañarte"
    ],
    "indagacion": [
        "alguna vez has considerado la posibilidad de estudiar algo mas","hay algun tema que siempre te haya llamado la atencion y que quizas te gustaria estudiar","te has preguntado como seria estudiar",
        "te gustaria estudiar en la cun","que opinas sobre la educacion continua o el aprendizaje a lo largo de la vida","crees que en algun momento te gustaria ampliar tus conocimientos a traves de estudios",
        "te ves volviendo a estudiar en el futuro","que factores te harian considerar seriamente la opcion de estudiar","hay algo que te gustaria aprender o profundizar si tuvieras la oportunidad",
        "que tipo de habilidades o conocimientos te gustaria adquirir","te interesa el desarrollo personal y profesional a traves de la educacion","has visto algun programa o curso que te haya parecido interesante",
        "que piensas sobre la idea de invertir tiempo y recursos en tu formacion","crees que estudiar podria abrirte nuevas puertas en el futuro","que te detendria de estudiar en este momento",
        "que necesitarias para tomar la decision de estudiar","te gustaria explorar opciones de estudio en algun momento","que tanto valoras la educacion en tu vida",
        "hay algun cambio en tu vida que podria motivarte a estudiar","que beneficios crees que podrias obtener al estudiar","estudiaste algo en el pasado",
        "como fue tu experiencia estudiando","que piensas sobre la gente que estudia actualmente","crees que la educacion es importante en general",
        "estas ocupado con otras cosas en este momento","estas ocupada con otras cosas en este momento","tienes planes a corto plazo que no incluyen estudiar",
        "estas enfocado en tu trabajo o en otras prioridades","estas enfocada en tu trabajo o en otras prioridades","sientes que necesitas estudiar algo en este momento",
        "hay algo mas que te interese hacer en lugar de estudiar","como te sientes con respecto a tu desarrollo profesional actual","crees que necesitas mas formacion para alcanzar tus metas",
        "que te parece la idea de volver a las aulas","te imaginas estudiando de nuevo alguna vez","que opinas sobre el sistema educativo actual",
        "crees que estudiar es para todos","que alternativas a la educacion formal consideras validas","te sientes realizada profesionalmente sin necesidad de mas estudios",
        "te sientes realizado profesionalmente sin necesidad de mas estudios","hay algun obstaculo importante que te impida estudiar","que necesitaria cambiar en tu vida para considerar estudiar",
        "te ves jubilando sin volver a estudiar","te sientes satisfecho con tu nivel actual de conocimientos o te gustaria seguir aprendiendo",
        "instagram con un","paga con una","indicacion","indicacion indicacion","indicacion","indicacion indicamense","indicaciones de mas",
        "validar","confirmar","corroborar","comprobar","verificar","asegurar","certificar",
        "cuina con la","diana con el","digan con ustedes","financiaciones precisamente","mis indicaciones de","de vacaciones de",
        "financiar con un","inducciones cuatro","inducciones y sabemos","de vacaciones","de vacaciones en","brindarte con la",
        "indicando con que","financiado no te","de vacaciones y","inducciones entonces","nacional tengo","nacional como",
        "indicio de a","nacional en","a ligacion del","nacional como","nacional pues","nacional tengo","nacional como",
        "nacional con","ingresa in direzione","nacional manifiestas","una accion bancaria","nacional como","nacional de",
        "nacional estos","nacional me","tiendi con informacion","ciudad con daniel","viajar con aguas","nacional el",
        "nacional nosotros","ingles con beca","nacional la","nacional y queria","naciones de","nacional estamos","nacional tengo",
        "nacional el","que necesitas","cual problema","como usas","que buscas","cuanto inviertes","por que cambiaste","que esperas",
        "cuando decides","quien decide","que opinas","ya conocias","con que frecuencia","que presupuesto","donde aplicas","que resultados",
        "como manejas","que prioridad","has considerado","te interesa","que impacto","carrera","descubrimos","crecer","carreras","homologar"
    ],
    "programas": [
        "ingenieria","derecho","administracion","programa","carrera","empresas","tecnica","profesional","administrativos","gestion","administrativa",
        "snies","diseño","grafico","graficos","publicaciones","sistemas","informatica","software","industrial","productivos","modas","vestuario","patronaje",
        "audiovisuales","comunicacion","social","deportiva","seguridad","salud","publicidad","mercadeo","agroindustrial","contaduria",
        "contable","financiera","comerciales","comunicativa","mercados","deportivo","territorial","territoriales"
    ],
    "argumentacion": [
        "beca","descuento","financiacion","convenio","modalidad virtual","modalidad presencial","modalidad plus","plataforma","flexibilidad","titulo",
        "perfil egresado","competencias","salidas laborales","costo","valor semestre","inscripcion","metodos de pago","requisitos admision","fechas inscripcion",
        "pasos admision","icfes","homologacion","homologar","documentos homologacion","proceso homologacion","estudio homologacion","contenido programatico",
        "materias homologadas","viabilidad homologacion","solicitud de homologacion","ruta academica post-homologacion","plan homologacion","comparar modalidades",
        "comparar perfiles","comparar financiacion","analizar elegibilidad","analizar materias","analizar diferencias","reingreso","requisito reingreso",
        "proceso reingreso","formulario reingreso","verificar pendientes","elegibilidad reingreso","actualizar informacion","actualizar plan","diferencias actuales",
        "costos actuales","beneficios","requisitos","proceso","titulo previo","modalidad","presencial","virtual","distancia","convenio"
    ],
    "objecion": [        
        "muy caro","no tengo tiempo","lo pensare","duda","que te preocupa exactamente","entiendo tu punto","podrias contarme mas",
        "que informacion te falta","que dudas tienes","que te impide decidir","comprendo tu inquietud sobre","validamos tu preocupacion","es normal tener preguntas",
        "que necesitas para sentirte seguro/a","inversion vs gasto","retorno de inversion (roi)","valor a largo plazo","opciones de financiacion detalladas","plan de pagos",
        "becas/descuentos aplicables","comparar valor","accesibilidad","presupuesto educativo","financiacion cun","flexibilidad de horarios","modalidad virtual/distancia","estudia a tu ritmo",
        "compatible con trabajo/familia","eficiencia (ciclos prop)","optimizar tiempo","gestion del tiempo","carga academica adaptable","metas profesionales","impacto en carrera","desarrollo personal",
        "oportunidad de mercado","costo de oportunidad (no actuar)","que cambiaria con el titulo","competitividad laboral","proyeccion futura","calidad cun (saber pro)","testimonios","alianzas","trayectoria",
        "soporte del asesor","proceso paso a paso","simplificamos el tramite","plataformas intuitivas (sinu/camiticket)","acompañamiento continuo","reconocimiento men","experiencia estudiantil",
        "que otras opciones consideras","como se compara la cun","programa alternativo cun","explorar otra modalidad","iniciar mas adelante (con seguimiento)","resolver dudas puntuales","estudio gratuito",
        "ahorro significativo vs empezar de cero","costo por credito homologado","financiacion flexible","inversion inteligente","valor del tiempo ahorrado","ahorro de tiempo significativo","avanzar semestres",
        "proceso agil","validacion rapida","menos tiempo de estudio total","proceso estandarizado","requisitos claros","respuesta garantizada","te guio en cada documento","simplificacion documental","soporte cun",
        "reconocimiento de tu esfuerzo previo","construir sobre lo existente","acelerar tu meta profesional","evitar redundancia","valor del titulo completo","optimizar tu formacion","experiencia con otras ies/sena",
        "transparencia","proceso definido","casos de exito homologacion","garantia de respuesta","claridad normativa","revisar estudio de homologacion","considerar otra carrera cun","aclarar dudas especificas del proceso",
        "consultar con coordinacion academica","evaluar plan de estudios detallado","inversion para finalizar","opciones de pago disponibles","presupuesto ajustado","valor de completar titulo","facilidades de pago reintegro",
        "costo vs beneficio final","plan de estudios adaptado","carga academica flexible","organizacion del tiempo","retomar sin perder mas tiempo","actualizacion al plan vigente","soporte academico","completar ciclo",
        "logro personal/profesional","beneficios de finalizar","actualizacion necesaria","mercado laboral actual","superar obstaculos previos","renovar compromiso","soporte para la solicitud","aclaramos requisitos",
        "experiencia previa (como mejoro)","confidencialidad","compromiso institucional","facilidad tramite online","respuesta formal","evaluar transferencia interna","discutir plan de estudios","plan de accion personalizado",
        "aclarar dudas sobre materias/creditos","ciudad","campo"
    ],
    "cierre": [
        "te inscribo","te llamo mañana","listo para matricularte","bienvenido","ciclos propedeuticos","3 titulos en 3 años","formacion para el futuro",
        "modalidad flexible","virtual","distancia","presencial entretenida","herramientas digitales","calidad academica","saber pro","portal de empleo",
        "servicios asistenciales","experiencia cun","bachillersitario","inscripcion facil","requisitos sencillos","plataforma sinu","camiticket",
        "admision directa","calendario academico","proceso guiado","te acompañamos","opciones de financiacion","credito educativo","icetex","sufi",
        "banco pichincha","facilidades de pago","inversion educativa","descuentos por convenio","alianzas financieras","becas","gran decision","excelente eleccion",
        "tu mejor version","unete al parche","futuro profesional","inicia ahora","oportunidad unica","empieza tu viaje","bienvenido a la cun","comunidad cunista",
        "confirmar inscripcion","enviar documentos","realizar pago","matricula","induccion","reunir papeles","formalizar admision","legalizar matricula",
        "validar conocimientos","reconocer tu titulo","ahorro tiempo y dinero","avanzar rapido","no empezar de cero","precios justos","hasta 50% homologable",
        "continuidad academica","mejorar perfil profesional","convenio sena","aprovechar estudios previos","estudio gratuito","respuesta rapida (5 dias)",
        "documentacion clara","tramite agil","homologacion en linea","te ayudo con los documentos","paso a paso","proceso simplificado","sin costo",
        "financiacion disponible","pago flexible","aplica financiacion","opciones de credito","facilidades de pago","buen camino","aprovecha tus estudios previos",
        "acelera tu carrera","felicitaciones por continuar","animate a validar","decision inteligente","continua tu formacion","preparar documentos",
        "enviar contenido tematico","enviar notas","esperar estudio","resultado homologacion","matricular asignaturas","formalizar homologacion","retomar estudios",
        "continuar tu formacion","segunda oportunidad","readmision","actualizar conocimientos","finalizar tu carrera","plan vigente","reincorporacion","culminar tu meta",
        "solicitud sencilla","formulario en linea","proceso claro","respuesta oportuna","apoyo en el tramite","fechas habilitadas","requisitos minimos","tramite guiado",
        "opciones de pago","financia tu regreso","planes de pago","credito disponible","facilidades","bienvenido de nuevo","nos alegra tu regreso","retoma con exito",
        "compromiso renovado","finaliza tu meta","estamos para apoyarte","completa tu ciclo","completar solicitud","enviar carta","enviar formulario","esperar aprobacion",
        "confirmar reintegro","realizar pago","inscribir materias","formalizar regreso"
    ],
}

# Requisitos mínimos y pesos para puntaje (ajústalo a tu rúbrica)
REQ_PESOS: Dict[str, Tuple[int, float]] = {
    "saludo":        (1, 0.10),
    "indagacion":    (1, 0.20),
    "programas":     (1, 0.20),
    "argumentacion": (1, 0.25),
    "objecion":      (1, 0.10),
    "cierre":        (1, 0.15),
}

UMBRAL_EFECTIVA = 0.80
# ==================================================


# ============== UTILIDADES DE TEXTO ================
def quitar_acentos(s: str) -> str:
    if not isinstance(s, str):
        return s
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def normalizar_texto(s: str) -> str:
    s = s.replace("\r", " ").replace("\n", " ")
    s = quitar_acentos(s.lower())
    s = re.sub(r"\s+", " ", s).strip()
    return s

def compilar_patrones(dic: Dict[str, List[str]]) -> Dict[str, List[re.Pattern]]:
    patrones: Dict[str, List[re.Pattern]] = {}
    for cat, frases in dic.items():
        pats = []
        for frase in frases:
            f = re.escape(normalizar_texto(frase))
            pats.append(re.compile(rf"(?<!\w){f}(?!\w)"))
        patrones[cat] = pats
    return patrones

def contar_por_categoria(texto_norm: str, patrones: Dict[str, List[re.Pattern]]) -> Dict[str, int]:
    res: Dict[str, int] = {}
    for cat, pats in patrones.items():
        total = 0
        for p in pats:
            total += len(p.findall(texto_norm))
        res[cat] = total
    return res

def segmentar_oraciones(texto: str) -> List[str]:
    if not texto:
        return []
    partes = re.split(r"[.!?¡¿;\n\r]+", texto)
    return [p.strip() for p in partes if p.strip()]

def contar_palabras(texto: str) -> int:
    if not texto:
        return 0
    return len(re.findall(r"\w+", texto, flags=re.UNICODE))
# ==================================================


# ====== EXTRACCIÓN DESDE NOMBRE / TEXTO ===========
def extraer_call_id_desde_archivo(nombre_txt: str) -> str:
    # Asume "<call_id>.txt" (ej: "000d90e3-bd95-....txt")
    return Path(nombre_txt).stem.strip()

def extraer_celular_desde_texto(texto: str) -> str:
    m = re.search(r"(\+?57)?\s?(\d{10})", texto)
    if m:
        return (m.group(1) or "") + (m.group(2) or "")
    m2 = re.search(r"\+?\d{7,12}", texto)
    return m2.group(0) if m2 else ""
# ==================================================


# ============== SENTIMIENTO (TextBlob opcional) ===
def sentimiento_textblob(texto: str) -> Tuple[float, float]:
    try:
        from textblob import TextBlob
        tb = TextBlob(texto)
        return float(tb.sentiment.polarity), float(tb.sentiment.subjectivity)
    except Exception:
        # Fallback sencillo
        texto_n = normalizar_texto(texto)
        pos = {"excelente", "bueno", "genial", "me gusta", "agradable", "perfecto", "gracias"}
        neg = {"malo", "pesimo", "caro", "no me gusta", "terrible", "problema", "queja"}
        ppos = sum(texto_n.count(w) for w in pos)
        pneg = sum(texto_n.count(w) for w in neg)
        pol = 0.0 if (ppos + pneg) == 0 else (ppos - pneg) / (ppos + pneg)
        return float(pol), 0.0
# ==================================================


# ============== NORMALIZACIÓN NÚMERO (10 dígitos) =
def numero10_desde_numero_vapi(num: str) -> str:
    """
    Devuelve los últimos 10 dígitos del número (remueve todo lo no numérico).
    Maneja prefijos +57, 57, 0057, espacios, guiones, etc.
    """
    if num is None:
        return ""
    digits = re.sub(r"\D+", "", str(num))
    if not digits:
        return ""
    # Últimos 10 dígitos (estándar Colombia)
    return digits[-10:]
# ==================================================


# ============== META DESDE SQL =====================
def cargar_meta_desde_sql() -> pd.DataFrame:
    import pyodbc
    conn = pyodbc.connect(SQL_CONNSTR)
    where_parts = []
    if META_DESDE:
        where_parts.append(f"started_at >= CONVERT(datetime2(7), '{META_DESDE}T00:00:00', 126)")
    if META_HASTA_EXC:
        where_parts.append(f"started_at <  CONVERT(datetime2(7), '{META_HASTA_EXC}T00:00:00', 126)")
    where_sql = ("WHERE " + " AND ".join(where_parts)) if where_parts else ""

    query = f"""
    SELECT
        call_id,
        number      AS numero_vapi,
        started_at  AS fecha,
        recording_url
    FROM dbo.vapi_entrenador
    {where_sql};
    """
    df = pd.read_sql(query, conn)
    conn.close()

    # Normaliza columnas y tipos
    df = df.rename(columns={"started_at": "fecha", "number": "numero_vapi"})
    df["call_id"] = df["call_id"].astype(str).str.lower().str.strip()
    df["numero_vapi"] = df["numero_vapi"].astype(str).str.strip()
    # numero10_vapi normalizado
    df["numero10_vapi"] = df["numero_vapi"].apply(numero10_desde_numero_vapi)
    # fecha a string estándar (opcional)
    try:
        df["fecha"] = pd.to_datetime(df["fecha"]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    except Exception:
        pass
    return df[["call_id", "numero_vapi", "numero10_vapi", "fecha", "recording_url"]]
# ==================================================


# ============== PIPELINE PRINCIPAL ===============
def analizar_txts_y_exportar(df_meta: Optional[pd.DataFrame] = None) -> pd.DataFrame:
    inp = Path(INPUT_TXT_DIR)
    outdir = Path(OUTPUT_DIR)
    outdir.mkdir(parents=True, exist_ok=True)

    # Recolecta .txt
    txt_files: List[Path] = []
    for dp, _, files in os.walk(inp):
        for f in files:
            if f.lower().endswith(".txt"):
                txt_files.append(Path(dp) / f)
    if not txt_files:
        print(f"❌ No encontré .txt en: {inp}")
        return pd.DataFrame()

    patrones = compilar_patrones(DICCIONARIOS_KEYWORDS)

    filas: List[dict] = []
    for p in txt_files:
        # Lee texto
        try:
            texto = p.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            texto = p.read_text(encoding="latin-1", errors="ignore")

        texto_norm = normalizar_texto(texto)

        # Conteos por categoría
        conteos = contar_por_categoria(texto_norm, patrones)

        # Puntaje
        puntaje = 0.0
        for cat, (min_req, peso) in REQ_PESOS.items():
            if conteos.get(cat, 0) >= min_req:
                puntaje += peso
        efectiva = "1" if puntaje >= UMBRAL_EFECTIVA else "0"

        # Sentimiento + métricas básicas
        polarity, subjectivity = sentimiento_textblob(texto)
        palabras   = contar_palabras(texto)
        oraciones  = len(segmentar_oraciones(texto))
        tamano_B   = p.stat().st_size if p.exists() else 0

        # call_id y celular desde texto
        call_id = extraer_call_id_desde_archivo(p.name)
        celular_txt = extraer_celular_desde_texto(texto)

        filas.append({
            "correo": None,
            "cedula_asesor": None,
            "asesor": None,
            "asesor_corto": None,
            "fecha": FECHA_ANALISIS_DEF,  # será reemplazada con meta si existe
            "saludo": conteos.get("saludo", 0),
            "indagacion": conteos.get("indagacion", 0),
            "programas": conteos.get("programas", 0),
            "argumentacion": conteos.get("argumentacion", 0),
            "objecion": conteos.get("objecion", 0),
            "cierre": conteos.get("cierre", 0),
            "puntaje": round(puntaje, 3),
            "efectiva": efectiva,
            "polarity": round(polarity, 3),
            "clasificacion": ("positivo" if polarity >= 0.1 else "negativo" if polarity <= -0.1 else "neutro"),
            "confianza": round(0.5 + abs(polarity)/2, 2),
            "palabras": palabras,
            "oraciones": oraciones,
            "archivo": p.name,
            "call_id": str(call_id),
            "numero_vapi": "",           # se llenará con meta
            "numero10_vapi": "",         # se calculará desde numero_vapi
            "tamano_B": tamano_B,
            "celular_texto": celular_txt,
            "subjectivity": round(subjectivity, 3),
            "tipo": TIPO,
            "recording_url": "",         # se llenará con meta
        })

    df_out = pd.DataFrame(filas)

    # --- normaliza call_id para merge ---
    df_out["call_id"] = df_out["call_id"].astype(str).str.lower().str.strip()

    # ========= MERGE con df_meta =========
    if df_meta is not None and not df_meta.empty:
        meta = df_meta.copy()

        # Asegura nombres esperados
        rename_map = {}
        if "started_at" in meta.columns and "fecha" not in meta.columns:
            rename_map["started_at"] = "fecha"
        if "number" in meta.columns and "numero_vapi" not in meta.columns:
            rename_map["number"] = "numero_vapi"
        if rename_map:
            meta = meta.rename(columns=rename_map)

        # Columnas útiles
        cols_keep = ["call_id", "fecha", "numero_vapi", "numero10_vapi", "recording_url"]
        meta = meta[[c for c in cols_keep if c in meta.columns]].copy()

        # Tipos limpios + normalización para join
        meta["call_id"] = meta["call_id"].astype(str).str.lower().str.strip()
        if "numero_vapi" in meta.columns:
            meta["numero_vapi"] = meta["numero_vapi"].astype(str).str.strip()

        # Si no viene numero10_vapi en meta, lo calculamos desde numero_vapi
        if "numero10_vapi" not in meta.columns and "numero_vapi" in meta.columns:
            meta["numero10_vapi"] = meta["numero_vapi"].apply(numero10_desde_numero_vapi)

        # Formatea fecha si viene como datetime
        if "fecha" in meta.columns:
            try:
                meta["fecha"] = pd.to_datetime(meta["fecha"]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
            except Exception:
                pass

        # MERGE
        df_out = df_out.merge(meta, on="call_id", how="left", suffixes=("", "_meta"))

        # Reemplaza fecha por la de meta si existe
        if "fecha_meta" in df_out.columns:
            df_out["fecha"] = df_out["fecha_meta"].combine_first(df_out["fecha"])
            df_out = df_out.drop(columns=["fecha_meta"])

        # Rellena numero_vapi, numero10_vapi y recording_url desde meta
        for c in ["numero_vapi", "numero10_vapi", "recording_url"]:
            if c + "_meta" in df_out.columns:
                df_out[c] = df_out[c].where(df_out[c].ne(""), df_out[c + "_meta"])
                df_out = df_out.drop(columns=[c + "_meta"])

    # Si quedó numero10_vapi vacío pero tenemos numero_vapi, lo calculamos aquí
    mask_vacios = (df_out["numero10_vapi"] == "") & (df_out["numero_vapi"] != "")
    df_out.loc[mask_vacios, "numero10_vapi"] = df_out.loc[mask_vacios, "numero_vapi"].apply(numero10_desde_numero_vapi)

    # ========= Orden y export =========
    columnas_salida = [
        "correo","cedula_asesor","asesor","asesor_corto","fecha",
        "saludo","indagacion","programas","argumentacion","objecion","cierre",
        "puntaje","efectiva","polarity","clasificacion","confianza",
        "palabras","oraciones","archivo","call_id",
        "numero_vapi","numero10_vapi","recording_url",
        "tamano_B","celular_texto","subjectivity","tipo"
    ]
    for c in columnas_salida:
        if c not in df_out.columns:
            df_out[c] = ""

    df_out = df_out[columnas_salida]

    base = f"Ventas_TXT_{datetime.now().strftime('%Y-%m-%d')}"
    xlsx = Path(OUTPUT_DIR) / f"{base}.xlsx"
    csv  = Path(OUTPUT_DIR) / f"{base}.csv"

    try:
        df_out.to_excel(xlsx, index=False)
        print(f"✅ Excel generado: {xlsx}")
    except Exception as e:
        print(f"⚠️ No se pudo escribir XLSX ({e}). Guardando CSV…")
        df_out.to_csv(csv, index=False, encoding="utf-8-sig")
        print(f"✅ CSV generado: {csv}")

    print(f"📊 Registros procesados: {len(df_out)} | Efectivas: {(df_out['efectiva']=='1').sum()}")
    print("🔎 Con numero_vapi:", (df_out["numero_vapi"] != "").sum(),
          "| Con numero10_vapi:", (df_out["numero10_vapi"] != "").sum())
    return df_out
# ================================================


# ============== MAIN =============================
if __name__ == "__main__":
    # 1) Carga META desde SQL
    try:
        df_meta = cargar_meta_desde_sql()
        print(f"ℹ️ df_meta: {len(df_meta)} filas",
              f"(rango {META_DESDE or '-inf'} .. {META_HASTA_EXC or '+inf'})")
    except Exception as e:
        print(f"❌ Error leyendo SQL: {e}")
        df_meta = pd.DataFrame()

    # 2) Ejecuta pipeline
    df_final = analizar_txts_y_exportar(df_meta=df_meta)

    # 3) Vista rápida
    if not df_final.empty:
        print(df_final.head(5))


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_3796\562065180.py:269: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


ℹ️ df_meta: 668 filas (rango -inf .. +inf)
✅ Excel generado: C:\Users\haider_bello\Downloads\audios_2\salidas\Ventas_TXT_2025-10-01.xlsx
📊 Registros procesados: 158 | Efectivas: 63
🔎 Con numero_vapi: 158 | Con numero10_vapi: 158
  correo cedula_asesor asesor asesor_corto                       fecha  \
0   None          None   None         None  2025-09-29 20:59:50.440000   
1   None          None   None         None  2025-09-29 21:01:15.509000   
2   None          None   None         None  2025-09-29 21:02:28.826000   
3   None          None   None         None  2025-09-29 21:03:56.469000   
4   None          None   None         None  2025-09-29 21:15:11.240000   

   saludo  indagacion  programas  argumentacion  objecion  ...  oraciones  \
0       2           0          0              0         0  ...          3   
1       2           0          0              0         0  ...          3   
2       2           0          0              0         0  ...          3   
3       2         